In [1]:
# import required packages
import re
import tensorflow as tf

In [2]:
# locate the saved models
dir_path  = './'
meta_file = 'my_model_final.ckpt.meta'
meta_path = ''.join([dir_path, meta_file])

In [3]:
# function to extract layer names
def get_layer_names(graph=None):
    """ Fetches layer names of a tensorflow model graph
    Args: 
        [Optional] graph: graph of the model
    Returns:
        A list of layer names of the model.

        example:
            layers, graph = get_layer_names(graph)

    """
    if graph is None:
        graph = tf.get_default_graph()
    L = []
    for i in graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
        v = re.search(r'\w+\/\w+\:\d', str(i)).group(0)
        v = v.split(":")[0]
        layerName = v.split("/")[0]
        L.append(layerName)
    return sorted(list(set(L))), graph

In [4]:
# function to extract variable values
def get_variable_values(layer, graph=tf.get_default_graph()):
    """ Fetches Variables of the given layer
    Args: 
        layer: layer of the model
    Returns:
        A Dictionary containing Kernels and Biases.

        example:
            layerWb, param_dict = get_variables(layer, graph)

    """

    M = []
    P = []
    param_dict = {}
    P_Wb = {}

    for i in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=layer):
        m = re.search(r'\w+\/\w+\:\d', str(i)).group(0)
        M.append(m)
        v = m.split(":")[0]
        param_type = ''.join(v.split("/")[1])
        P.append(param_type)
        print(("saving: {}".format(m)))
    param_dict[P[0]] = graph.get_tensor_by_name(M[0]).eval()  # Weights
    param_dict[P[1]] = graph.get_tensor_by_name(M[1]).eval()  # biases
    return param_dict

In [5]:
# restore the model and get layer names and respective variable values.
tf.reset_default_graph()
params = dict()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(meta_path)
    saver.restore(sess, tf.train.latest_checkpoint(dir_path))
    layers, g = get_layer_names()
    print(layers)
    params.update({layer: get_variable_values(layer, g) for layer in layers})

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
['hidden1', 'hidden2', 'outputs']
saving: hidden1/kernel:0
saving: hidden1/bias:0
saving: hidden2/kernel:0
saving: hidden2/bias:0
saving: outputs/kernel:0
saving: outputs/bias:0


In [6]:
W1 = params['hidden1']['kernel']
b1 = params['hidden1']['bias']

In [7]:
W2 = params['hidden2']['kernel']
b2 = params['hidden2']['bias']

In [8]:
W3 = params['outputs']['kernel']
b3 = params['outputs']['bias']